# COSI MEASURE Control

This notebook is a minimal working example and Installation guide.


## Installation

This Quick-Start-Guide is a short summary with minor changes of https://www.klipper3d.org/Installation.html

### 1. Download and Install Klipper

     git clone https://github.com/Klipper3d/klipper
     ./klipper/scripts/install-octopi.sh

### 2.1 Setting things up!

     cd klipper
     make menuconfig 

Opens a menu. Set
- Enable extra low-level configuration options
- Micro-controller Architecture (STMicroelectronics STM32)
- Processor model (STM32F446)
- Bootloader offset (32KiB bootloader)
- Clock Reference (12 MHz crystal)
- Communication interface (USB (on PA11/PA12))
- USB ids (doesnt matter. But choose something)
- no GPIO pins to set at startup ()

### 2.2 Config File

Get a ```printer.cfg```-file. 

Save it as a file named "printer.cfg" in the home directory of the pi user (ie, /home/cosi/printer.cfg).

More Infos: https://www.klipper3d.org/Installation.html#obtain-a-klipper-configuration-file


### 3. Going into Bootloader Mode

1. Power On
2. Press and hold "boot"
3. Press "reset"
4. release "boot"

### 4. Searching Device ID:

     lsusb

Will show up something like

     Bus 002 Device 001: ID 1d6b:0003 Linux Foundation 3.0 root hub
     Bus 001 Device 031: ID 0483:df11 STMicroelectronics STM Device in DFU Mode
     Bus 001 Device 002: ID 2109:3431 VIA Labs, Inc. Hub


### 5. Compiling and Downloading to MCU

Replace 0483:df11 with your hardware's ID from step 4 And execute the following command:

     cd klipper
     make flash FLASH_DEVICE=0483:df11


this will do a lot of things and eventually will end like the following

     Downloading to address = 0x08008000, size = 29236
     Download	[=========================] 100%        29236 bytes
     Download done.
     File downloaded successfully
     Transitioning to dfuMANIFEST state
     dfu-util: can't detach
     Resetting USB to switch back to runtime mode
     cosi@raspberrypi:~/klipper $ 


### 6. getting new Serial Port

     ls /dev/serial/by-id/*


will return something similar to 

     /dev/serial/by-id/usb-Klipper_stm32f446xx_280058000951363131343032-if00

Copy and paste it in /home/cosi/printer.cfg to your MCU serial definition:

     [mcu]
     serial: /dev/serial/by-id/usb-Klipper_stm32f446xx_280058000951363131343032-if00


### 7. Start Klipper

     sudo service klipper start


## Troubleshooting

- If Klipper can not connect to MCU, perform step 3, 4 and 5 of this quick start guide.
- no more known workaraounds
- Klipper only supports one endstop per axis. Workaround: Macro for the other pin.


Hint: After changing your printer.cfg file you need to restart klipper.

Enjoy your measurements!




In [3]:
import serial
import time
from datetime import datetime
import os

os.system('sudo service klipper stop')

time.sleep(2)

os.system('sudo service klipper start')

time.sleep(1)

ser = serial.Serial('/tmp/printer', 250000)
time.sleep(1)

def command(command):
  #start_time = datetime.now()
  global ser
  command = command + "\r\n"

  ser.write(str.encode(command)) 
  #time.sleep(1)

  while True:
    line = ser.readline()
    print(line)

    if line == b'ok\n':
      break



In [4]:
command("G28 X0 Y0 Z0") # homing procedure

b'// Klipper state: Ready\n'


b'ok\n'


In [3]:
# set actual position --> no homing
command("G92 X0 Y0 Z0")

b'ok\n'


In [23]:
command("M18") # disable motor --> looses homing

b'ok\n'


In [38]:
# hard disable
command("hard_disable_drives")

b'ok\n'


In [3]:
# hard enable
command("hard_enable_drives")

b'ok\n'


In [9]:
command("G0 X100 Y80 Z300")




b'ok\n'


In [27]:
command("QUERY_ENDSTOPS")

b'x:TRIGGERED y:TRIGGERED z:TRIGGERED\n'
b'ok\n'


In [28]:
command("G0 X0 Y0 Z0") # moves X-axis to x=10
time.sleep(1)
command("G0 X0 Y0 Z10")
time.sleep(1)
command("G0 X0 Y0 Z0")


b'ok\n'
b'!! Must home axis first: 0.000 0.000 10.500 [0.000]\n'
b'ok\n'
b'ok\n'


In [29]:
# some further examples. mostly useless for COSI Measure



command(ser, "G28 X0 Y0 Z0\r\n")
# command(ser, "G28 X0 Y0\r\n")
# command(ser, "G28 X0\r\n")
# command(ser, "G28 Y0\r\n")
# command(ser, "G28 Z0\r\n")

# Extruder Temp
# command(ser, "M104 S190 T0\r\n") #  start heating T0 to 190 degrees Celsius
# command(ser, "G28\r\n") # Home
# command(ser, "M109 S190 T0\r\n") # wait for T0 to reach 190 degrees before continuing with any other commands

# Bed Temp
# command(ser, "M140 S55\r\n") # heat bed to 50 degrees celsius but do not wait
# command(ser, "G28\r\n") # Home
# command(ser, "M190 S55\r\n") # wait for bed to heat to 50 degrees celsius and wait

# Fan
# command(ser, "M106 S255\r\n") # fan speed full
# command(ser, "M106 S127\r\n") # fan speed about half
# command(ser, "M106 S0\r\n") # turn off fan

# Set Units(does not seem to work on ender 5)
# command(ser, "G20\r\n") # inches
command(ser, "G21\r\n") # millimeters

# Absolute Mode
command(ser, "G90\r\n")

# Relative Mode
# command(ser, "G91 X10\r\n")

# Move
# command(ser, "G0 X7 Y18\r\n") # rapid motion but does not extrude material
command(ser, "G0 X350 Y350\r\n") # rapid motion but does not extrude material ender 5 plus is 350 x 350
command(ser, "G1 Z0.345 F500\r\n") # change layer
command(ser, "G0 X50 Y50\r\n") # rapid motion but does not extrude material ender 5 plus is 350 x 350

time.sleep(2)
ser.close()

TypeError: command() takes 1 positional argument but 2 were given

In [9]:
# slile
command("G1 Z100")
command("G1 X80 Y150")
command("G1 Z48 ")
command("G1 X80 Y170")
command("G1 Z55")

command("G1 X100 Y150")
command("G1 Z48")
command("G1 X100 Y170")
command("G1 Z55")

command("G1 X40 Y170")
command("G1 Z48")
command("G1 X80 Y200")
command("G1 X100 Y200")
command("G1 X130 Y170")
command("G1 Z55")
command("G1 X10 Y10")


b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'


In [23]:
command("G0 X50 Y0")
for i in range (2):
    command("G0 Z55")
    command("G0 X150 Y100")
    command("G0 Z45")
    command("G0 X150 Y150")
    command("G0 Z55")
    command("G0 X250 Y150")
    command("G0 Z45")
    command("G0 X250 Y100")
    command("G0 Z55")
command("G0 X50 Y0")

b'ok\n'
b'ok\n'
b'ok\n'


b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'


In [3]:
for i in range (1):
    command("G0 X497")  #max:497
    command("G0 Y460")  #max:460
    command("G0 Z610")  #max:610
    command("G4 P1000")
    command("G0 X10 Y10 Z10") 


b'ok\n'
b'ok\n'


b'ok\n'
b'ok\n'
b'ok\n'


In [7]:
#command("G0 X50 Y50 F500")
#command("G0 X100 Y100 F1000")
#command("G0 X250 Y250 F2000")
command("G4 P5000")
command("G0 X497 Y460 Z400 F4000")
command("G4 P5000")
command("G0 X10 Y10 Z10 F4000")


b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'


In [5]:
command("G0 Z200 F2000")

b'ok\n'


In [11]:
for i in range (5):
    command("G4 P1000")
    command("G1 F4000")
    command("G1 X400 Y400 Z400")
    command("G4 P5000")
    command("G1 X0 Y0 Z0")
    #command("G4 P5000")
    #command("G0 X10 Y10 Z10") 
command("G1 F2000")




b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'


b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'


In [21]:
command("G0 Z10 F2000")

b'ok\n'


In [7]:
command("G0 Z50")
command("G0 X76 Y10")
command("G0 Z45")
command("G2 X10 Y75  I11 J79")
command("G0 Z50")
command("G0 X10 Y84")
command("G0 Z45")
command("G2 X75 Y154 I80 J-9")
command("G0 Z50")
command("G0 X84 Y154")
command("G0 Z45")
command("G2 X154 Y84 I-10 J-79")
command("G0 Z50")
command("G0 X154 Y75")
command("G0 Z45")
command("G2 X84 Y10 I-80 J11")
command("G0 Z50")

b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'


b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'


In [23]:
command("G0 X200 Y0 ")
command("G0 Z45")

command("G2 X200 Y0 I0 J200 F2000")
command("G2 X200 Y0 I0 J100 F2000")
command("G2 X200 Y0 I0 J50 F2000")
command("G2 X200 Y0 I0 J25 F2000")
command("G2 X200 Y0 I0 J12.5 F2000")

command("G0 Z200")

b'ok\n'
b'ok\n'


b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'


In [4]:
#halbkreis
command("G0 X50 Y100 Z50")

command("G0 Z45")
command("G2 X150 Y100 I50 J0")
command("G0 X140 Y100")
command("G0 Z50")

command("G0 X50 Y100")

command("G0 Z45")
command("G0 X60 Y100 ")
command("G2 X140 Y100 I40 J0")
command("G0 Z50")

command("G0 X75 Y100 Z50")
command("G0 Z45")
command("G0 X75 Y80 ")
command("G0 X70 Y80 ")
command("G0 X70 Y100 ")
command("G0 X75 Y100 ")
command("G0 Z50")

command("G0 X125 Y100")
command("G0 Z45")
command("G0 X125 Y80")
command("G0 X130 Y80")
command("G0 X130 Y100")
command("G0 X125 Y100")
command("G0 Z50")
command("G0 X200 Y200")

b'ok\n'
b'ok\n'


b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'


In [51]:
command("G0 X10 Y75")   #1

command("G0 X15 Y75")


command("G0 X10 Y84")   #2

command("G0 X15 Y84")


command("G0 X43 Y38")   #3

command("G0 X50 Y46")


command("G0 X38 Y43")   #4

command("G0 X46 Y50")


command("G0 X75 Y10")   #5

command("G0 X75 Y14")


command("G0 X84 Y10")   #6

command("G0 X84 Y14")



command("G0 X109 Y46")   

command("G0 X116 Y38")


command("G0 X113 Y50")  

command("G0 X121 Y43")



command("G0 X144 Y75")   

command("G0 X154 Y75")


command("G0 X144 Y84")   

command("G0 X154 Y84")



command("G0 X109 Y113")   

command("G0 X116 Y121")


command("G0 X113 Y109")   

command("G0 X121 Y116")



command("G0 X75 Y144")   #5

command("G0 X75 Y154")


command("G0 X84 Y144")   #6

command("G0 X84 Y154")



command("G0 X43 Y121")   #3

command("G0 X50 Y113")


command("G0 X38 Y116")   #4

command("G0 X46 Y109")



command("G0 X56 Y106")   

command("G0 X63 Y98")


command("G0 X96 Y98")   

command("G0 X103 Y106")



command("G0 X75 Y10")

command("G2 X10 Y75 I75 J75")

command("G0 X10 Y84")

command("G2 X75 Y154 I75 J75")

command("G0 X84 Y154")

command("G2 X154 Y84 I75 J75")

command("G0 X154 Y75")

command("G2 X84 Y10 I75 J75")


command("G0 X75 Y14")

command("G2 X14 Y75 I65 J65")

command("G0 X14 Y84")

command("G2 X75 Y144 I65 J65")

command("G0 X84 Y144")

command("G2 X144 Y84 I65 J65")

command("G0 X144 Y75")

command("G2 X84 Y14 I65 J65")



command("G0 X38 Y43")

command("G2 X38 Y116 I55 J55")

command("G0 X43 Y121")

command("G2 X116 Y121 I55 J55")

command("G0 X121 Y116")

command("G2 X121 Y43 I55 J55")

command("G0 X116 Y38")

command("G2 X43 Y38 I55 J55")



command("G0 X46 Y50")

command("G2 X46 Y109 I45 J45")

command("G0 X50 Y113")

command("G2 X109 Y113 I45 J45")

command("G0 X113 Y109")

command("G2 X113 Y50 I45 J45")

command("G0 X109 Y46")

command("G2 X50 Y46 I45 J45")



command("G0 X103 Y106")

command("G2 X56 Y106 I35 J35")

command("G0 X96 Y98")

command("G2 X63 Y98 I25 J25")

b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'


b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'!! Move out of range: -0.454 96.171 200.000 [0.000]\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'ok\n'
b'!! Move out of range: -0.079 102.132 200.000 [0.000]\n'
b'ok\n'
b'ok\n'


KeyboardInterrupt: 